In [ ]:
from string import ascii_uppercase, ascii_lowercase

key_lookup = {
    '`': 'grave_accent_and_tilde', '~': '!Sgrave_accent_and_tilde',
    '!': '!S1', '@': '!S2', '#': '!S3', '$': '!S4', '%': '!S5', '^': '!S6', '&': '!S7', '*': '!S8', '(': '!S9', ')': '!S0',
    '-': 'hyphen', '_': '!Shyphen', '=': 'equal_sign', '+': '!Sequal_sign', 'del': 'delete_or_backspace',
    '[': 'open_bracket', '{': '!Sopen_bracket', ']': 'close_bracket', '}': '!Sclose_bracket', '\\': 'backslash', '|': '!Sbackslash',
    ';': 'semicolon', ':': '!Ssemicolon', "'": 'quote', '"': '!Squote', 'ret': 'return_or_enter',
    ',': 'comma', '<': '!Scomma', '.': 'period', '>': '!Speriod', '/': 'slash', '?': '!Sslash',
    'iterm': 'f9'
}

def express(char):
    if char in ascii_uppercase: return f'!S{char.lower()}'
    if char in key_lookup: return key_lookup[char]
    return char

# all non-modifier keys (not including f-keys)
keys_all = list(ascii_lowercase) + [str(i) for i in range(10)] + ['tab', '`', '-', '=', 'del', ';', "'", 'ret', ',', '.', '/']

In [ ]:
boilerplate = """
:templates {{:delay-set-var "sleep 0.1 && /Library/Application\\\\ Support/org.pqrs/Karabiner-Elements/bin/karabiner_cli --set-variables '{{\\"%s\\": %d}}'"}}
:main [
{}
]
"""
def generate(keys_used, layers):
    ins = str()
    unused = [key for key in keys_all if key not in keys_used]
    for layer in layers:
        # base layer
        ins += f'    {{:des "{layer["desc"]}",\n' +\
               f'    :rules [\n' +\
                '            ' + ' '.join([f'[:{express(k)} :vk_none]' for k in unused]) + '\n' +\
                f'       [:condis ["{layer["name"]}" 0] ' + ' '.join([f'["{layer["name"]}_{n+1}_held" 0]' for n in range(len(layer["modifiers"]))]) + ']\n' +\
                '            ' + ' '.join(
                             [f'[:{express(k)} :{express(m)}]' for k, m in zip(keys_used, layer['base'])]
                ) + ']}\n'
        for n, mod in enumerate(layer['modifiers']):
            ins += f'    {{:des "{mod["desc"]}",\n' +\
                   f'    :rules [\n' +\
                   f'        [:{mod["activator"]} [["{layer["name"]}_{n+1}_held" 1] ["{layer["name"]}" {n+1}]] nil ' +\
                                 f'{{:afterup [["{layer["name"]}_{n+1}_held" 0] [:delay-set-var "{layer["name"]}" 0]]}}]\n' +\
                   f'        [:condis ["{layer["name"]}_{n+1}_held" 1]]\n' +\
                    '            ' + ' '.join(
                                 [f'[:{express(k)} :{express(m)}]' for k, m in zip(keys_used, mod['map'])]
                    ) + '\n' +\
                   f'        [:condis ["{layer["name"]}" {n+1}]]\n' +\
                    '            ' + ' '.join(
                                 [f'[:{express(k)} [:{express(m)} ["{layer["name"]}" 0]]]' for k, m in zip(keys_used, mod['map'])]
                    ) + ']}\n'
    return boilerplate.format(ins)

In [ ]:
goku = generate(
    keys_used = [
        'tab', 'q', 'w', 'e', 'r', 't',     'y', 'u', 'i', 'o', 'p', '[', 'del',
               'a', 's', 'd', 'f', 'g',     'h', 'j', 'k', 'l', ';', "'",
               'z', 'x', 'c', 'v', 'b',     'n', 'm', ',', '.', '/', 'ret'
    ],
    layers = [
        {
            'name': 'layer_prim', 'desc': 'primary layer: base',
            'base': [
                'tab', 'q', 'w', 'e', 'r', 't',     'y', 'u', 'i', 'o', 'p', 'del', 'iterm',
                       'a', 's', 'd', 'f', 'g',     'h', 'j', 'k', 'l', ';', "'",
                       'z', 'x', 'c', 'v', 'b',     'n', 'm', ',', '.', '/', 'ret'
            ],
            'modifiers': [
                {
                    'desc': 'primary layer: caps', 'activator': 'left_shift',
                    'map': [
                        'tab', 'Q', 'W', 'E', 'R', 'T',     'Y', 'U', 'I', 'O', 'P', 'del', 'iterm',
                               'A', 'S', 'D', 'F', 'G',     'H', 'J', 'K', 'L', ':', '"',
                               'Z', 'X', 'C', 'V', 'B',     'N', 'M', '<', '>', '?', 'ret'
                    ],
                }, {
                    'desc': 'primary layer: numchar', 'activator': 'right_shift',
                    'map': [
                        'tab', '7', '8', '9', '!', '@',     '#', '*', '[', ']', '+', 'del', 'iterm',
                               '4', '5', '6', '|', '\\',    '%', '-', '(', ')', '=', '~',
                               '1', '2', '3', '^', '$',     '&', '_', '{', '}', '`', 'ret'
                    ],
                }
            ]

        }
    ]
)
print(goku)